In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class SolverNoLineal:
    def __init__(self,G,seed):
        self.G = G
        self.dim = len(G)
        self.h = 1e-8
        self.itmax = 1000
        self.error = 1e-10
        self.epochs=int(1e5)
        self.seed = seed
    def GetVector(self,r):
        v = np.zeros(self.dim)
        for i in range(self.dim):
            v[i] = self.G[i](r)
        return v
    def Jacobian(self,r):
        J = np.zeros((self.dim,self.dim))
    
        for i in range(self.dim):
            for j in range(self.dim):
                delta = np.zeros(self.dim)
                delta[j] = self.h
                J[i,j] = ( self.G[i](np.add(r,delta)) - self.G[i](np.add(r,-delta)) )/(2*self.h)
        
        return J.T
    def Metric(self):
        vec = self.GetVector(self.sol)
        return np.linalg.norm(vec)
    def NewtonRaphson(self):
        self.sol = self.seed
        self.it = 0
        self.residuo = 1
        while self.residuo > self.error and self.it < self.itmax:
            self.it += 1
            rc = self.sol
            J = self.Jacobian(self.sol)
            F = self.GetVector(self.sol)
            inv_J = np.linalg.inv(J)
            self.sol = rc - np.dot(inv_J,F)
            self.residuo = self.Metric()
        if self.it < self.itmax:
            return self.sol
        else:
            return False
    def GradientDescent(self,lr):
        self.sol = self.seed
        self.it = 0
        self.residuo = 1
        while self.residuo > self.error and self.it < self.epochs:
            CF = self.Metric()
            self.it += 1
            rc = self.sol
            J = self.Jacobian(rc)
            F = self.GetVector(rc)
            Grad = np.dot(J,F)
            self.sol = rc - lr*Grad
            NF = self.Metric()
            self.residuo = np.abs(CF**2 - NF**2)/NF**2
        if self.it < self.epochs:
            return self.sol
        else:
            return False


In [3]:
G1=(lambda r: np.log(r[0]**2 + r[1]**2) - np.sin(r[0]*r[1]) - np.log(2) - np.log(np.pi), \
   lambda r: np.exp(r[0] - r[1]) + np.cos(r[0]*r[1]))

In [4]:
sist1 = SolverNoLineal(G1,np.array([2,2]))

In [5]:
r1_newton = sist1.NewtonRaphson()
print(r1_newton,sist1.it)

[1.77245385 1.77245385] 110


In [6]:
r1_grad = sist1.GradientDescent(1e-1)
print(r1_grad,sist1.it)

[1.77245385 1.77245385] 148


In [7]:
G2=(lambda r: 6*r[0] - 2*np.cos(r[1]*r[2]) - 1, \
   lambda r: 9*r[1] + np.sqrt(r[0]**2 + np.sin(r[2]) + 1.06) + 0.9, \
    lambda r: 60*r[2] + 3*np.exp(-r[0]*r[1]) + 10*np.pi - 10 )

In [8]:
sist2 = SolverNoLineal(G2,np.array([0,0,0]))

In [9]:
r2_newton = sist2.NewtonRaphson()
print(r2_newton,sist2.it)

[ 0.49879961 -0.20587945 -0.41233964] 12


In [10]:
r2_grad = sist2.GradientDescent(1e-4)
print(r2_grad,sist2.it)

[ 0.49879961 -0.20587945 -0.41233964] 8546
